### ✨ Notebook'u Doğru Çalıştırma Adımları ✨

**1. Önce Ortamı Sıfırlayın:** Başlamadan önce, menüden `Runtime > Restart runtime` seçeneği ile Colab ortamını yeniden başlatın. Bu, olası çakışmaları önler.
**2. Hücreleri Sırayla Çalıştırın:** Notebook'taki hücreleri **yukarıdan aşağıya doğru, sırayla** çalıştırın. Bir hücrenin çalışması tamamen bitmeden bir sonrakine geçmeyin.
    - **Hücre 1 (Kurulum):** Bu hücre, interaktif arayüzün çalışması için kritik olan `output.enable_custom_widget_manager()` komutunu içerir. **Mutlaka ilk bu hücreyi çalıştırın.**
    - **Hücre 2 (MLflow):** Deney takibini başlatır.
    - **Hücre 3 (Eğitim):** Bu hücreyi çalıştırdığınızda, model seçim kutuları ve "Eğitimi Başlat" butonu ekranda belirecektir.

### ⚠️ ÖNEMLİ: İnteraktif Arayüzü Etkinleştirme

**Lütfen ilk olarak bu hücreyi çalıştırın.**

Bu hücre, butonlar ve seçim kutuları gibi interaktif arayüz elemanlarının Colab'da doğru bir şekilde görüntülenmesini sağlar. Eğer bu hücreyi çalıştırmazsanız veya çalıştırmadan önce başka bir hücre çalıştırırsanız, arayüz yüklenmeyebilir.

*Sorun devam ederse, `Runtime > Restart runtime` menüsünden Colab'ı yeniden başlatıp bu hücreyi tekrar çalıştırmayı deneyin.*

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# 🚀 İnteraktif JetX Model Eğitmeni (v4.4 - Kararlı Sürüm)

## 🛠️ 1. Sistem Kurulumu ve Projeyi Yükleme

Bu hücre, projeyi GitHub'dan klonlar (veya günceller), gerekli tüm kütüphaneleri yükler ve Colab arayüzü için gerekli eklentileri etkinleştirir.

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        print("🔌 Enabling ipywidgets extension for Google Colab...")
        from google.colab import output
        output.enable_custom_widget_manager()
        print("✅ ipywidgets extension enabled.")
    
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

setup_environment()
print("🎉 Kurulum tamamlandı!")

## 📊 2. MLflow Deney Takibini Başlatma

In [ ]:
from getpass import getpass
from colab.health_check import ColabHealthCheck
import time

NGROK_AUTH_TOKEN = getpass('Lütfen ngrok authtokeninizi girin: ')

health_checker = ColabHealthCheck()
ngrok_ok, error_message = health_checker.check_ngrok_tunnel(NGROK_AUTH_TOKEN)

if ngrok_ok:
    print("✅ ngrok bağlantı testi başarılı.")
    get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')
    time.sleep(5)
    print("\n✅ MLflow arayüzü için public URL oluşturuluyor...")
    print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000, "http").public_url
        print(f"--> {public_url}")
    except Exception as e:
        print(f"❌ ngrok tüneli oluşturulamadı. Hata: {e}")
else:
    print(f"\n❌ ngrok ön kontrolü başarısız oldu. Lütfen authtoken'ınızı ve internet bağlantınızı kontrol edin.")
    print(f"   Hata Detayı: {error_message}")

## 🚀 3. Modelleri Seç ve Eğitimi Başlat

In [ ]:
import torch
import ipywidgets as widgets
from IPython.display import display, HTML
from src.training.pipeline_manager import PipelineManager
from src.config.settings import get_all_models
import traceback

def run_training_interface():
    """
    Handles the entire process of creating and displaying the training UI.
    It fetches models, creates widgets, defines callbacks, and displays the UI.
    This makes the process more robust and easier to debug.
    """
    output_area = widgets.Output()

    def on_button_clicked(b, model_checkboxes, output_area):
        """
        This function is executed when the 'Start Training' button is clicked.
        It captures the selected models and initiates the training pipeline.
        """
        with output_area:
            output_area.clear_output()
            
            selected_models = [cb.description for cb in model_checkboxes if cb.value]
            
            if not selected_models:
                print("⚠️ Lütfen en az bir model seçin!")
                return

            print(f"🚀 Seçilen modellerle eğitim başlatılıyor: {', '.join(selected_models)}")
            
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            if device == 'cuda':
                print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
            else:
                print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")
            
            try:
                pipeline_manager = PipelineManager(models_to_train=selected_models, device=device)
                pipeline_manager.run_pipeline()
            except Exception as e:
                print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
                traceback.print_exc()

    try:
        print("🔍 Eğitilebilir modeller alınıyor...")
        available_models = get_all_models()
        
        if not available_models:
            with output_area:
                print("❌ UYARI: `get_all_models()` fonksiyonu boş bir liste döndürdü. `config.yaml` dosyasında modellerin tanımlı olduğundan emin olun.")
            display(output_area)
            return
        
        print(f"✅ Başarıyla {len(available_models)} model bulundu: {available_models}")

        print("🎨 Arayüz elemanları oluşturuluyor...")
        model_checkboxes = [widgets.Checkbox(value=False, description=model) for model in available_models]
        start_button = widgets.Button(description='Eğitimi Başlat', button_style='success', icon='rocket')
        
        start_button.on_click(lambda b: on_button_clicked(b, model_checkboxes, output_area))

        print("✅ Checkbox, button ve output alanı başarıyla oluşturuldu.")
        
        print("🖥️ Arayüz oluşturuluyor ve görüntüleniyor...")
        
        ui_container = widgets.VBox([
            widgets.HTML("<h3>Eğitmek istediğiniz modelleri seçin:</h3>"),
            *model_checkboxes,
            start_button,
            output_area
        ])
        
        display(ui_container)
        
        with output_area:
            print("✅ Arayüz başarıyla görüntülendi. Lütfen model seçimi yapın.")

    except Exception as e:
        with output_area:
            output_area.clear_output()
            print(f"❌ HATA: Arayüz oluşturulurken kritik bir hata oluştu: {e}")
            traceback.print_exc()
        display(output_area)

# --- Arayüzü Başlat ---
run_training_interface()